In [236]:
import pandas as pd 
import fastparquet #make sure that this is installed in your anaconda environment
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
from scipy import stats
import array

#loads the file into the label "data"
filename='C:\\Users\james\OneDrive\Documents\Capstone\BR00121436.parquet' #edit the filename here for testing other .parquet files

data=pd.read_parquet(filename, engine='fastparquet') 

data_numerical=data._get_numeric_data() #removing columns that have non quantitiative data


In [237]:
df_normalized = (0.6745 * (data_numerical - data_numerical.median(axis=0))) / stats.median_abs_deviation(data_numerical, axis=0)
df_normalized.shape

(383, 4762)

In [238]:
df_normalized=df_normalized.dropna(axis=1) #get rid of data ruined by normalization (not numeric)
df_normalized.shape

(383, 4655)

In [239]:
a = np.array([1,3,67])
b = np.array([50,5,7])

a = np.vstack([a,b])     #this took me like 20 mins to figure out
a

array([[ 1,  3, 67],
       [50,  5,  7]])

In [240]:
df_normalized_row = np.array([df_normalized.loc[0]]) 
rows_to_keep = np.empty([1,df_normalized.shape[1]])
print(df_normalized_row.shape)
print(rows_to_keep.shape)


(1, 4655)
(1, 4655)


In [241]:
#remove outliers

n_MAD = 500
threshold = 0.6745 * n_MAD

rows_to_keep = np.empty([1,df_normalized.shape[1]])
for x in range(0,df_normalized.shape[0]-1): #parse through each individual row of new data set
    df_normalized_row = np.array([df_normalized.loc[x]])
    if (df_normalized_row<threshold).all(): #check if any values are above threshold, if not, add to kept rows
        rows_to_keep = np.vstack([rows_to_keep,df_normalized_row])  #appending rows to new array

rows_to_keep.shape #thank god I spent an hour trying to figure it out when Ryan and Matt had it in 2 lines of code


(320, 4655)

In [242]:
rows_to_keep = (np.abs(df_normalized) < threshold).all(axis=1)
rows_to_keep

df_normalized = df_normalized[rows_to_keep]

df_normalized

,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,0.929365,0.784549,-2.220039,-1.092584,-2.148888,-1.194359,-2.271343,-1.218634,-0.210349,0.867531,...,0.042161,0.006028,0.000000,-0.000899,-0.002324,0.002622,0.005457,0.005176,0.001382,0.008923
1,-0.049885,0.067561,-0.377407,-0.365741,-0.179743,-0.337579,-0.267565,-0.358505,-0.567844,1.458828,...,0.494492,0.446222,0.467337,0.468439,0.467843,0.474551,0.480343,0.477963,0.468037,0.482412
2,-0.255167,-0.287798,-0.842309,-0.816953,-0.460167,-0.793607,-0.664639,-0.858712,-0.589570,0.024406,...,0.511576,0.495855,0.485560,0.477594,0.488176,0.485942,0.482623,0.472389,0.474296,0.480659
3,0.368541,0.469587,-0.541298,1.404656,-0.413842,1.400328,-0.491083,1.525419,0.023701,0.759922,...,1.181923,1.140685,1.157824,1.147435,1.154626,1.142659,1.150119,1.143066,1.136766,1.141057
4,0.198631,0.259520,1.076719,-0.794426,1.138373,-0.799529,1.155722,-0.838874,-0.287378,0.726641,...,0.415107,0.351498,0.392457,0.392668,0.390491,0.393588,0.401507,0.389012,0.390491,0.390869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,-1.035483,-0.877881,-1.433231,0.386943,-1.029663,0.733066,-1.262222,0.618528,0.161959,0.465937,...,-0.790639,-0.814473,-0.814817,-0.803564,-0.812272,-0.805204,-0.804155,-0.805100,-0.799760,-0.801417
378,0.581987,0.641068,1.243222,1.458987,1.367530,1.221997,1.356889,1.451025,0.452300,0.489234,...,-1.190778,-1.178653,-1.183336,-1.172446,-1.181848,-1.178716,-1.178380,-1.188936,-1.179440,-1.189258
380,0.093420,0.106845,-0.663400,0.209373,-0.497228,0.262561,-0.596269,0.264274,0.403910,0.205234,...,-1.477795,-1.480980,-1.460343,-1.455341,-1.463234,-1.460881,-1.461658,-1.479138,-1.462693,-1.477909
381,-0.523032,-0.471119,-1.112631,0.057644,-0.853008,0.321786,-1.022269,0.214678,0.249851,0.264031,...,-1.788487,-1.772183,-1.772263,-1.762748,-1.774573,-1.769139,-1.769972,-1.785737,-1.769225,-1.784629


In [264]:
#remove highly correlated features
#you will need to pip install feature engine in the conda environment!!

#code from sam

from sklearn.datasets import make_classification
from feature_engine.selection import DropCorrelatedFeatures

#set the function to use later
corrs = DropCorrelatedFeatures(variables=None, method='spearman', threshold=0.90) #this is taken right from ryan

#df_normalized.shape
dropped_correlated = corrs.fit_transform(df_normalized)
dropped_correlated

,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_Orientation,...,Nuclei_Texture_InfoMeas1_Mito_3_00_256,Nuclei_Texture_InfoMeas1_Mito_5_01_256,Nuclei_Texture_InfoMeas1_RNA_10_01_256,Nuclei_Texture_InfoMeas1_RNA_3_00_256,Nuclei_Texture_InfoMeas1_RNA_3_01_256,Nuclei_Texture_InfoMeas2_AGP_10_01_256,Nuclei_Texture_InfoMeas2_AGP_3_00_256,Nuclei_Texture_InfoMeas2_DNA_3_03_256,Nuclei_Texture_InfoMeas2_ER_3_00_256,Nuclei_Texture_InfoMeas2_Mito_3_00_256
0,0.929365,-2.220039,-1.092584,-2.148888,-1.194359,-0.210349,0.867531,-0.0437,0.033104,0.911135,...,0.592628,0.939164,0.009961,-0.617242,-0.505875,-0.209019,0.100914,-1.166689,0.083469,-0.920982
1,-0.049885,-0.377407,-0.365741,-0.179743,-0.337579,-0.567844,1.458828,0.0000,-0.307594,0.490427,...,-0.355829,-0.388037,-0.674500,-0.890935,-1.134757,0.380910,0.207628,-1.141577,0.482399,0.305860
2,-0.255167,-0.842309,-0.816953,-0.460167,-0.793607,-0.589570,0.024406,-1.1058,0.187591,0.694911,...,0.148000,0.215703,-0.351480,-0.284000,-0.670427,0.008938,0.103814,-1.384659,0.088992,-0.798823
3,0.368541,-0.541298,1.404656,-0.413842,1.400328,0.023701,0.759922,-1.0013,-1.046923,-0.049014,...,0.183268,0.513579,-0.779802,-0.357290,-0.689978,0.245804,0.201828,-0.829188,0.114769,-0.202257
4,0.198631,1.076719,-0.794426,1.138373,-0.799529,-0.287378,0.726641,-0.5548,-0.257938,0.603021,...,0.217906,0.043369,-0.366184,-0.435161,-0.632955,-0.275025,-0.245905,-0.389733,0.104336,0.046699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,-1.035483,-1.433231,0.386943,-1.029663,0.733066,0.161959,0.465937,1.1894,0.291042,0.645684,...,1.067486,1.052722,0.562083,-0.472952,0.362503,-1.650153,-2.551269,-2.329360,-1.507345,-0.561619
378,0.581987,1.243222,1.458987,1.367530,1.221997,0.452300,0.489234,0.0418,-0.697949,-0.667670,...,0.040936,-0.521568,1.063927,1.248226,1.091582,-1.470699,-1.129773,-0.945204,-0.920610,0.671098
380,0.093420,-0.663400,0.209373,-0.497228,0.262561,0.403910,0.205234,0.3059,-0.024828,0.611906,...,-0.770857,-0.779498,1.735106,0.499290,1.336781,-1.875330,-1.520090,-0.588618,-1.322609,0.313282
381,-0.523032,-1.112631,0.057644,-0.853008,0.321786,0.249851,0.264031,-1.4003,0.045518,-1.407645,...,-0.857768,1.001593,2.186670,1.629565,2.509824,-3.371469,-4.808495,-1.043140,-4.008334,-2.231015


In [261]:
new_columns = list(dropped_correlated) #headers of new data table
new_columns

['Cells_AreaShape_Area',
 'Cells_AreaShape_BoundingBoxMaximum_X',
 'Cells_AreaShape_BoundingBoxMaximum_Y',
 'Cells_AreaShape_BoundingBoxMinimum_X',
 'Cells_AreaShape_BoundingBoxMinimum_Y',
 'Cells_AreaShape_Compactness',
 'Cells_AreaShape_Eccentricity',
 'Cells_AreaShape_EulerNumber',
 'Cells_AreaShape_Extent',
 'Cells_AreaShape_Orientation',
 'Cells_AreaShape_Solidity',
 'Cells_AreaShape_Zernike_1_1',
 'Cells_AreaShape_Zernike_2_0',
 'Cells_AreaShape_Zernike_2_2',
 'Cells_AreaShape_Zernike_3_1',
 'Cells_AreaShape_Zernike_3_3',
 'Cells_AreaShape_Zernike_4_0',
 'Cells_AreaShape_Zernike_4_2',
 'Cells_AreaShape_Zernike_4_4',
 'Cells_AreaShape_Zernike_5_1',
 'Cells_AreaShape_Zernike_5_3',
 'Cells_AreaShape_Zernike_5_5',
 'Cells_AreaShape_Zernike_6_0',
 'Cells_AreaShape_Zernike_6_2',
 'Cells_AreaShape_Zernike_6_4',
 'Cells_AreaShape_Zernike_6_6',
 'Cells_AreaShape_Zernike_7_1',
 'Cells_AreaShape_Zernike_7_3',
 'Cells_AreaShape_Zernike_7_5',
 'Cells_AreaShape_Zernike_7_7',
 'Cells_AreaShape_

In [265]:
search_item1 = 'Intensity'

todrop = []
column2 = []

for x in range(0,len(new_columns)):
    if search_item1 in new_columns[x]: #determining if the query item is found in the remaining column headers
        column2.append(new_columns[x])
    else:
        todrop = todrop + [x]    #adds index values of columns that did not contain query string to array todrop

dropped_correlated1 = dropped_correlated.drop(dropped_correlated.columns[todrop],axis = 1)   #creates new data set without column indexes in todrop
dropped_correlated1


,Cells_Intensity_IntegratedIntensityEdge_AGP,Cells_Intensity_IntegratedIntensityEdge_DNA,Cells_Intensity_IntegratedIntensityEdge_ER,Cells_Intensity_IntegratedIntensityEdge_Mito,Cells_Intensity_IntegratedIntensity_AGP,Cells_Intensity_IntegratedIntensity_DNA,Cells_Intensity_IntegratedIntensity_ER,Cells_Intensity_LowerQuartileIntensity_AGP,Cells_Intensity_LowerQuartileIntensity_DNA,Cells_Intensity_LowerQuartileIntensity_ER,...,Nuclei_Intensity_MassDisplacement_AGP,Nuclei_Intensity_MassDisplacement_DNA,Nuclei_Intensity_MassDisplacement_ER,Nuclei_Intensity_MassDisplacement_Mito,Nuclei_Intensity_MassDisplacement_RNA,Nuclei_Intensity_MeanIntensityEdge_DNA,Nuclei_Intensity_MeanIntensity_Mito,Nuclei_Intensity_MinIntensityEdge_ER,Nuclei_Intensity_StdIntensityEdge_DNA,Nuclei_Intensity_StdIntensityEdge_ER
0,0.898312,-1.323392,0.278161,-0.211912,0.871008,-0.730277,0.325162,0.383587,-0.569219,0.031180,...,-1.051716,-0.797423,-0.927907,-0.624155,-0.772337,-1.268430,-0.546147,0.121064,-1.531291,-1.240006
1,0.849020,-1.209984,0.784980,0.687052,0.656206,-1.001911,0.587468,1.049725,-0.214058,0.737126,...,-0.463536,0.018211,-0.307924,-0.449039,-0.143187,-0.872114,0.317018,0.768207,-1.073667,0.180077
2,1.068835,-1.043532,1.013637,-0.379706,0.536413,-1.082692,0.423678,1.149019,-0.099777,0.822138,...,-0.919408,-0.776060,-0.953599,-0.678761,-0.939235,-0.794266,-0.588931,0.860970,-1.168717,-0.537862
3,1.866832,1.472925,2.549939,0.499137,1.288218,1.350627,1.924984,1.438190,0.976714,1.493441,...,-0.703334,-0.530915,-0.731648,-0.386505,-0.198206,0.607540,-0.101033,1.907153,-0.162433,0.303288
4,1.054181,0.235961,1.183733,1.205941,0.648535,0.368245,0.739170,0.827795,0.535494,0.808281,...,-0.808086,-0.143935,-0.792807,-0.794316,-0.671277,0.396316,0.534033,0.907195,-0.238741,-0.539442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,-1.668198,-1.444573,-1.187054,0.663055,-1.778013,-2.132538,-1.810150,-0.958445,-0.425669,-0.674500,...,-2.514228,-1.434100,-0.986434,-1.321350,-2.205822,-1.267341,0.565219,-0.994298,-0.984417,-1.446937
378,-0.469472,-0.723887,-0.201490,2.001349,-0.674500,-1.218065,-0.631588,-0.700630,-0.275741,-0.316330,...,-1.205377,-1.435501,-0.189742,-0.614245,-1.097156,-1.369142,1.263689,-0.573875,-1.494699,-1.108107
380,-1.349133,-0.586701,-0.928211,0.251230,-1.500659,-1.275765,-1.647569,-1.305450,-0.108427,-0.675566,...,-0.637089,-0.892679,0.880033,0.449535,0.083334,-1.197115,-0.053609,-1.687665,-1.727862,-1.076515
381,-3.357979,-1.004205,-2.405048,-5.279522,-3.219070,-1.621522,-3.154919,-2.801126,-0.090778,-1.499836,...,-3.476157,-2.244131,-0.810854,-0.267184,-2.233447,-0.743638,-3.142340,-2.653977,-1.519912,-3.298021


In [273]:
new_headers = list(dropped_correlated1) #list of headers from filtered dataset
categories = []

for x in range(0,len(new_headers)):
    split_header = new_headers[x].split('_')  #splits up the headers by "_". We can offer list of things to filter by
    if split_header[0] in categories:         #user picks from this list to filter down the data. Then run again to filter further
        split_header[0]                      # would come before step above
    else:
        categories.append(split_header[0])
    
categories

['Cells', 'Cytoplasm', 'Image', 'Nuclei']